In [2]:
import numpy as np
import pandas as pd
import os

Willkommen zum fünften Python-Problem Set in diesem Modul!  
Sie finden hier zwei Aufgaben (*Exercise 9* und *Exercise 10*) zu Inhalten aus Kapitel 4 *Text Mining* - konkret zu den Teilen 4.1 *Einführung und Grundlagen* und 4.2 *Topic Analysis und Topic Mining*.

# Exercise 9 -- Einführung und Grundlagen in Text Mining

Das Ziel dieser *Exercise* ist die Heranführung an die grundlegenden Schritte des Text Mining. Hierzu gehört zunächst das **Einlesen von Textdaten** und die **Konvertierung der Textdaten** in maschinenlesbare Form, in unserem Fall als Document Term Matrix (DTM).  

Die DTM eines Dokumentencorpus beinhaltet in den Zeilen die entsprechenden Dokumente des Corpus, während die Spalten alle verschiedenen Wörter des gesamten Corpus darstellen. Die Einträge der DTM beschreiben dabei das Vorkommen eines Wortes innerhalb eines Dokumentes. Informationen zum *i*-ten Dokument des Corpus erhalten wir daher in der *i*-ten Zeile der DTM. Sie entspricht dem im Skript eingeführten **Dokumentvektor** des Vektorraummodells. In der folgenden *Infobox* finden Sie eine kurze Wiederholung zu Dokumentvektoren. 

## Info: Wiederholung Dokumentvektoren

Einträge von Nullen im Dokumentvektor beschreiben, dass die entsprechenden Wörter, die in den Spalten dargestellt werden, nicht im Dokument vorhanden sind. Einträge größer als 0 bedeuten hingegen, dass die entsprechenden Wörter im Dokument enthalten sind. Welche genauen Werte die positiven Einträge annehmen können, hängt von der gewählten Ausprägung des Vektormodells ab. Bei einer **binären Repräsentation** können Einträge nur die Werte 0 oder 1 annehmen. Diese Art der Repräsentation wird bevorzugt für Sentiment Analysis verwendet, da das Vorkommen eines einzelnen Wortes in diesem Kontext wichtiger sein kann als z.B. die Anzahl des Vorkommens (Manning et al. (2008)). Möchte man die Anzahl der Wörter innerhalb eines Dokumentes berücksichtigen wählt man für die Repräsentation der Werte in der DTM die **Termfrequenz (tf)**. Eine weitere Repräsentation der Einträge der DTM ist die **invertierte Termfrequenz (tf-idf)**, bei der die Anzahl des Vorkommens von Wörtern innerhalb eines Dokument normiert mit der Anzahl des Vorkommens von den Wörtern über die Dokumente betrachtet wird.

***


Im Folgenden werden wir die DTM mit den drei verschiedenen Repräsentationsmethoden von Dokumenten, die Sie im Skript kennengelernt haben (**binäre Repräsentation**, **Termfrequenz** und **invertierte Termfrequenz**), für eine Sammlung von Textdokumenten erstellen und betrachten. Die hierzu verwendeten Textdaten wurden erstmals von Pang und Lee (2004) für eine Sentiment Analysis benutzt. Der verwendete Datensatz besteht aus 1.000 positiven und 1.000 negativen Filmkritiken, wobei jede Filmkritik in einer separaten Datei gespeichert ist. Die Filmkritiken, die eine positive Grundstimmung gegenüber einem Film ausdrücken, liegen im Ordner *"pos"*, wohingegen Filmkritiken, die eine negative Grundstimmung ausdrücken, im Ordner *"neg"* abgelegt sind. Die Einteilung der Filmkritiken in die Klassen *positiv* und *negativ* wird allerdings erst für die Verwendung von Klassifizierungsalgorithmen eine Bedeutung spielen.

Führen Sie den folgenden Code aus, um die Daten in R einzulesen, sie für die spätere Erstellung einer DTM aufzubereiten und beispielhaft die Eigenschaften und den Inhalt eines Elements aus jedem der beiden Datensätze einsehen zu können.  
*Hinweis: Der Code in diesem Problem Set ist relativ rechenintensiv. Wundern Sie sich daher nicht, falls Sie nach der Eingabe eines korrekten Codes einige Sekunden warten müssen.*

In [3]:
# {"9"}
#Laden des Packages 'tm', um Methoden des Text Mining verwenden zu können

#Separates Einlesen der Datensätze für positiv und negativ bewertete Filme
pos_movie_dir = 'pos'
neg_movie_dir = 'neg'
#Funktion zum Einlesen der Textdateien

def read_txt_in_dir(directory):
    dir_txt = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename)) as file:
            txt = file.read()
            dir_txt.append(txt)
    return dir_txt
#Speichern der Daten jeweils in einem VCorpus ("Volatile Corpus")
pos_movies = read_txt_in_dir(pos_movie_dir)
neg_movies = read_txt_in_dir(neg_movie_dir)
#Alle Daten in einem  Vektor zusammenfassen
all_movies = pd.DataFrame({'pos_movies': pos_movies, 'neg_movies': neg_movies})
#Details über das dritte Element der Datensätze anzeigen


#Ausgabe des dritten Elements der Datensätze
print(pos_movies[2])
#- - -
print(neg_movies[2])

you've got mail works alot better than it deserves to . 
in order to make the film a success , all they had to do was cast two extremely popular and attractive stars , have them share the screen for about two hours and then collect the profits . 
no real acting was involved and there is not an original or inventive bone in it's body ( it's basically a complete re-shoot of the shop around the corner , only adding a few modern twists ) . 
essentially , it goes against and defies all concepts of good contemporary filmmaking . 
it's overly sentimental and at times terribly mushy , not to mention very manipulative . 
but oh , how enjoyable that manipulation is . 
but there must be something other than the casting and manipulation that makes the movie work as well as it does , because i absolutely hated the previous ryan/hanks teaming , sleepless in seattle . 
it couldn't have been the directing , because both films were helmed by the same woman . 
i haven't quite yet figured out what i like

Wie Sie sehen, wurden die in den Ordnern *"pos"* und *"neg"* enthaltenen Dateien in Objekten der Klasse *"VCorpus"* gespeichert. Hierbei handelt es sich um Listen, deren Elemente jeweils zwei Spalten beinhalten: *meta* und *content*. Während die Spalte *meta* Informationen über das Dokument enthält, wie beispielsweise den Dateinamen im Feld *id*, gibt die Spalte *content* den Inhalt des Textdokumentes zeilenweise aus. Entsprechend  erfolgen der Zugriff auf Informationen über einzelne Elemente der Datensätze (*...$meta*) und das Anzeigen des Inhaltes (*...$content*) jeweils mithilfe von doppelten eckigen Klammern *[[]]*, da in R der Zugriff auf Listenelemente allgemein über doppelte eckige Klammern erfolgt.

## a) Termfrequenz

Um nun mit den Daten weiterarbeiten zu können, sollen diese in Form einer DTM dargestellt werden. Konkret wollen wir hier als Gewichte die Termfrequenzen verwenden. Hierzu bietet das package "tm" den Befehl **DocumentTermMatrix(x, control = list(stopwords, removeNumbers, removePunctuation, tolower, stripWhitespace, stemming))**. Dabei nimmt die Funktion für *x* ein Objekt der Klasse *"Vcorpus"* entgegen, welches die Textdokumente beinhaltet. Die Inputvariablen im Argument *control* stellen boolesche Ausdrücke dar:  
* Mithilfe von *stopwords* kann festgelegt werden, ob die im Text enthaltenen Stoppwörter entfernt werden sollen. Stoppwörter sind Wörter ohne inhaltliche Relevanz, wie im Deutschen z.B. "und","so","der","die" oder "das". Wenn *stopwords* auf *FALSE* gesetzt wird, bleiben diese in der DTM enthalten.  
* Durch *TRUE* bei *removeNumbers* werden alle im Text enthaltenen Zahlen entfernt.  
* *removePunctuation* entfernt, falls es auf *TRUE* gesetzt wird, die Interpunktion im Text.  
* Die Variable  *tolower* wandelt alle Buchstaben in Kleinbuchstaben um, wenn sie auf *TRUE* gesetzt wird.  
* Mittels *TRUE* beim Befehl *stripWhitespace* wird sichergestellt, dass überflüssige aufeinanderfolgende Leerzeichen zu einem Leerzeichen komprimiert werden.  
* Durch die Variable *stemming* kann festgelegt werden, ob Wörter im Text auf ihren Wortstamm reduziert werden sollen (*TRUE*) oder nicht (*FALSE*). Das Komprimieren von Wörtern auf ihre Stämme dient vor allem der Dimensionsreduktion und kann je nach Anwendungsfall sinnvoll sein oder nicht. Für das Bilden der Wortstämme wird intern das R-Package *"SnowballC"* verwendet, welches auf den Porter-Stemming-Algorithmus zurückgreift.

Per Default sind alle Befehle auf *FALSE* gesetzt und die DTM wird mit der absoluten Termfrequenz **tf** dargestellt.  
- - -
*Aufgabe*: Wenden Sie den Befehl *DocumentTermMatrix()* auf den gesamten Datensatz *full_corpus* (ein *VCorpus*-Objekt) an, sodass Stoppwörter, Zahlen und Interpunktion entfernt werden, alle Wörter in Kleinbuchstaben konvertiert werden, unnötige Leerzeichen entfernt werden und **keine** Wortstämme gebildet werden. Bitte beachten Sie auch, die Parameter in der genannten Reihenfolge anzugeben. Geben Sie anschließend die erzeugte DTM mit dem Befehl *print()* aus.
- - -

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

pos_movies_vectorized = vectorizer.fit_transform(pos_movies)
neg_movies_vetorized = vectorizer.fit_transform(neg_movies)

In [5]:
# {"9_2"}
#Wenden Sie den oben beschriebenen Befehl an und speichern Sie das
#Ergebnis in der Variable DTM_tf
DTM_tf <- DocumentTermMatrix(x=full_corpus,
                          control=list(
                            stopwords=TRUE,
                            removeNumbers=TRUE,
                            removePunctuation=TRUE,
                            tolower=TRUE,
                            stripWhitespace=TRUE,
                            stemming=FALSE))

#Geben Sie anschließend das Ergebnis für die DTM_tf 
#mit dem Befehl print() aus
print(DTM_tf)

NameError: name 'DTM_tf' is not defined